A notebook for training a music generation transformer

In [3]:
import sys
sys.path.insert(0, '../src/libs')

In [4]:
# our file imports
from music_transformer import *
from transformer_training_helpers import *
from song_classification import *
from dataset import *
from torch.autograd import Variable

# musicautobot
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.utils.midifile import *
from musicautobot.utils.file_processing import process_all

In [ ]:
# FIXME: DONT NEED
#midi_path = '../src/mid_data_collections/mid_0_to_10000'
# numpy_path = '../numpy_path'

In [ ]:
# FIXME: DONT NEED
# numpy_files = get_files(numpy_path, extensions='.npy', recurse=True); len(numpy_files)
#midi_files = get_files(midi_path, '.mid', recurse=True)
#print(midi_files[0])
#print(len(midi_files))

NEW DATASET CODE

In [5]:
myDs = MidiDataset('../numpy_path', 150, 'N/A', '.npy')
dataloader = DataLoader(myDs, 1, shuffle=True)

for _, item in enumerate(dataloader):
    print(item)
    break

tensor([[[  0.,   1.,  81.,  ..., 138.,  77., 139.],
         [ 71., 139.,  68.,  ..., 138.,  68., 138.],
         [  8., 139.,  90.,  ..., 140.,  69., 153.],
         ...,
         [ 74., 139.,  74.,  ..., 142.,   8., 141.],
         [ 75., 138.,  69.,  ..., 138.,   8., 138.],
         [ 64., 138.,   8.,  ..., 139.,  67., 139.]]])


END NEW DATASET CODE

In [ ]:
# FIXME: DONT NEED
# data_path = './saved_data/'
# data_save_name = 'music_item_data_0-10000.pkl'
# def create_databunch(files, path):
#     #save_file.parent.mkdir(exist_ok=True, parents=True)
#     vocab = MusicVocab.create()
#     processors = [OpenNPFileProcessor(), MusicItemProcessor()]

#     #data = MusicDataBunch.from_files(midi_files, path, processors=processors, encode_position=True)
#     data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)
#     return data

In [ ]:
# FIXME: DONT NEED
# all_data = create_databunch(numpy_files, data_path); all_data

In [ ]:
# the_train_data = data_gen(all_data.train_ds)
# print(the_train_data[0].src.shape)
# print(the_train_data[0].trg.shape)
# print(the_train_data[0].trg_y.shape)
# print(the_train_data[0].src_mask.shape)
# print(the_train_data[0].trg_mask.shape)

In [ ]:
# for _, batch in enumerate(dataloader):
#     #out, w2 = model.forward(batch.src, batch.trg, 
#     #                    batch.src_mask, batch.trg_mask)
#     print(batch.shape)
#     print('beg')
#     batch = batch.long()
#     batch = batch.squeeze(0)
#     PAD = 0
#     trg = batch[:,:-1]
#     trg_y = batch[:, 1:]
#     src_mask = (batch != PAD).unsqueeze(-2)
#     trg_mask = (trg != PAD).unsqueeze(-2)
#     trg_mask = trg_mask & Variable(subsequent_mask(trg.size(-1)).type_as(trg_mask.data))
#     print(batch.shape)
#     print(trg.shape)
#     print(trg_y.shape)
#     print(src_mask.shape)
#     print(trg_mask.shape)
#     break

In [6]:
def run_epoch(data_iter, model, loss_compute):#, bad_idxs):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    i = 0
    collection_of_losses = []
    print(len(data_iter))
    for _, batch in enumerate(data_iter):
        #out, w2 = model.forward(batch.src, batch.trg, 
        #                    batch.src_mask, batch.trg_mask)
        #print('ITERATION: ', _)
        try:
            batch = batch.cuda()
            batch = batch.long()
            batch = batch.squeeze(0)
            PAD = 0
            trg = batch[:,:-1]
            trg_y = batch[:, 1:]
            src_mask = (batch != PAD).unsqueeze(-2)
            trg_mask = (trg != PAD).unsqueeze(-2)
            trg_mask = trg_mask & Variable(subsequent_mask(trg.size(-1)).type_as(trg_mask.data))
            ntokens = (trg_y != PAD).data.sum()
            
            out, w2 = model.forward(batch, trg, src_mask, trg_mask)
            #print("HEREEE")
            loss = loss_compute(out, trg_y, ntokens.item())

            total_loss += loss
            total_tokens += ntokens
            tokens += ntokens
            if i % 200 == 1:
                collection_of_losses.append(torch.div(loss,ntokens).cpu())
                elapsed = time.time() - start
                print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                        (i, torch.div(loss,ntokens), tokens / elapsed))
                start = time.time()
                tokens = 0
        except:
            continue
        
        i += 1
    return (total_loss / total_tokens), collection_of_losses

In [7]:
# My version of Greedy Decoding
def test_my_model():

  #---------- TRAIN -----------
  #V = len(all_data.vocab) # 312
  V = 312
  criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
  model = make_model(V, V, N=2, d_model=128)
  model.cuda() # uncomment for GPU
  model_opt = NoamOpt(model.src_embed[0].d_model, 1, 400, torch.optim.Adam(model.parameters(), lr=0.6, betas=(0.9, 0.98), eps=1e-9))
  #the_train_data = data_gen(all_data.train_ds)
  the_train_data = dataloader
  collection_of_losses = np.array([])
  for epoch in range(10):
    print("epoch: ", epoch)
    model.train()
    totalLoss_div_totalTokens, collection_of_losses_inp = run_epoch(the_train_data, model, SimpleLossCompute(model.generator, criterion, model_opt))#, bad_idxs)
    collection_of_losses = np.concatenate([collection_of_losses, collection_of_losses_inp])
    #model.eval()
    #print(run_epoch(the_train_data, model, SimpleLossCompute(model.generator, criterion, None), bad_idxs))
  return model, collection_of_losses

model, losses = test_my_model()

C:\Users\Trevi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
c:\Users\Trevi\Desktop\DRAGN\MusGen\src\libs\music_transformer.py:269: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


epoch:  0
9636
Epoch Step: 1 Loss: 5.828012 Tokens per Sec: 8372.560547
Epoch Step: 201 Loss: 3.438637 Tokens per Sec: 75299.812500
Epoch Step: 401 Loss: 2.700613 Tokens per Sec: 78180.859375
Epoch Step: 601 Loss: 2.301977 Tokens per Sec: 75996.578125
Epoch Step: 801 Loss: 1.975284 Tokens per Sec: 76136.804688
Epoch Step: 1001 Loss: 1.915180 Tokens per Sec: 75636.804688
Epoch Step: 1201 Loss: 1.601505 Tokens per Sec: 76770.390625
Epoch Step: 1401 Loss: 2.178509 Tokens per Sec: 79151.289062
Epoch Step: 1601 Loss: 2.015445 Tokens per Sec: 73008.843750
Epoch Step: 1801 Loss: 1.636420 Tokens per Sec: 59001.003906
Epoch Step: 2001 Loss: 1.542135 Tokens per Sec: 73755.734375
Epoch Step: 2201 Loss: 1.041046 Tokens per Sec: 73224.398438
Epoch Step: 2401 Loss: 1.551455 Tokens per Sec: 75923.046875
Epoch Step: 2601 Loss: 1.337303 Tokens per Sec: 74112.867188
Epoch Step: 2801 Loss: 1.608479 Tokens per Sec: 75081.070312
Epoch Step: 3001 Loss: 1.570492 Tokens per Sec: 76832.281250
Epoch Step: 3201 

Save model to a .pt file

In [8]:
torch.save(model, '../models/full_ds_epochs_15_LM.pt')

Load the Model from a previous state

In [ ]:
myModel = torch.load("./epochs_30_LM.pt")